### Bob möchte nun die Satoshis mittels Segwit-Transaktion an ein spezielles von ihm erstelltes Skript binden. 
Das Skript ist der Satz: **"We make it visible"**. Auch dazu eignet sich die **P2WSH** Transaktion.

### Aufgrund der Tatsache, dass wir mit einer "leeren" Blockchain beginnen, müssen zunächst ein paar Vorarbeiten geleistet werden.
    1. Verbindung zur Blockchain in regtest Modus aufbauen
    2. Anlegen zweier Nodes und Wallets (Alice und Bob)
    3. Mining 101 Blöcke damit Alice Wallet Coins enthält
    4. Funding der Transaktion welche später als Input für die P2SH-Locking Transaktion genutzt wird.
    5. Bob erzeugt das von ihm gewüsnchte Locking Skript

In [9]:
import sys
sys.path.insert(0, "/media/henning/Volume/Programming/Bitcoin/bitcoin/test/functional")
from test_framework.test_shell import TestShell
from io import BytesIO
from decimal import Decimal

from src.transaction import CTx, CTxIn, CTxOut
from src.script import Script, p2wsh_script, p2pk_script
from src.ecdsa import PrivateKey
from src.crypto import sha256
from src.helper import string_to_hex

In [10]:
test = TestShell().setup(num_nodes=2, setup_clean_chain=True)
alice = test.nodes[0]
bob = test.nodes[1]

alice.createwallet("alice", descriptors=False)
bob.createwallet("bob", descriptors=False)

alice_address_0 = alice.getnewaddress("", "legacy")
test.generatetoaddress(alice, 101, alice_address_0)

alice_address_1 = alice.getnewaddress("", "legacy")
alice_private_key_wif_format = alice.dumpprivkey(alice_address_1)
alice_private_key = PrivateKey.convert_wif_format(alice_private_key_wif_format)
alice_private_key_int_format = alice_private_key.get_private_key_int()
alice_public_key = alice_private_key.get_public_key()

alice_p2wsh_funding_tx = alice.sendtoaddress(alice_address_1, 0.00011)
alice_p2wsh_funding_tx_parsed = alice.decoderawtransaction(alice.gettransaction(alice_p2wsh_funding_tx)["hex"])
test.generatetoaddress(alice, 1, alice_address_0)

2024-03-10T16:39:26.312000Z TestFramework (INFO): PRNG seed is: 6908410086429393061
2024-03-10T16:39:26.313000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_4vj6m9t2


['7f72f77cb712102bbfc5f5eb7bfb99f3a551f0a5cc886b6580983b7f99f2ae0f']

**1.** Zunächst muss Bob das von ihnen gewünschte Script erzeugen.

In [11]:
bob_secret = "We make it visible"
bob_secret_hex = string_to_hex(bob_secret)
bob_original_script = Script([bytes.fromhex(bob_secret_hex), 0x87])
bob_original_script_hex = bob_original_script.serialize_script().hex()[2:]  
bob_original_script_sha256 = sha256(bytes.fromhex(bob_original_script_hex))

**1.1** In der ersten Transaktion werden von Alice die 10000 Satoshis an das P2WSH Script von Bob versendet. Um das notwendige Transaktionsformular ausfüllen zu können, benötigt Alice folgende Informationen.

In [12]:
alice_previous_tx_id_to_spent = alice_p2wsh_funding_tx_parsed["txid"]

for tx_out in alice_p2wsh_funding_tx_parsed["vout"]:
    if tx_out["value"] == Decimal("0.00011000"):
        alice_previous_tx_index_to_spent = tx_out["n"]
        alice_previous_script_pub_key_to_spent = tx_out["scriptPubKey"]["hex"]

#Transaktionsoutputdaten
alice_amount_to_spent = 10000

**1.2** Nun kann Alice die Lockingtransaktion erzeugen. Um die Satoshis an Bob's Skript zu senden, muss Alice im Output der Transaktion ein scriptPubKey der Form: OP_00 <script_hash>.

In [13]:
transaction_input = CTxIn(bytes.fromhex(alice_previous_tx_id_to_spent), alice_previous_tx_index_to_spent, script_sig=Script())

alice_script_pubkey = p2wsh_script(bob_original_script_sha256)
transaction_output = CTxOut(alice_amount_to_spent, alice_script_pubkey)

alice_locking_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Alice unsignierte P2WSH Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

Alice unsignierte P2WSH Lockingtransaktion: 0100000001ae8c7047cff55469574f4e39f9691ac5c2c70d1d80d5166ad028b0269422f9330100000000ffffffff01102700000000000022002009b415a9b6f470f1460585bcccc0db3e9a2418aa93ed0420a42822ac7a12e383ffffffff


**1.3** Nun muss Alice die Lockingtransaktion noch signieren. D.h. sie muss nachweisen, dass sie aus der vorhergehenden Transaktion berechtigt ist die 10000 Satoshis auszugeben.

In [14]:
alice_script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(alice_previous_script_pub_key_to_spent)//2)[2:]}{alice_previous_script_pub_key_to_spent}")))
alice_locking_transaction.sign_transaction(0, [alice_private_key_int_format], alice_script_sig)

print(f"Alice signierte P2WSH Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

alice_p2wsh_locking_tx = alice.sendrawtransaction(alice_locking_transaction.serialize_transaction().hex())
test.generatetoaddress(alice, 1, alice_address_0)

Alice signierte P2WSH Lockingtransaktion: 0100000001ae8c7047cff55469574f4e39f9691ac5c2c70d1d80d5166ad028b0269422f933010000006a47304402200f7afbc9e3bb346b6c2ddf8eba994ad50f9bbae515437f5ee61ed53b8784da9b02201661c0541cfa8426c1704acf9a3a1f170366bfd4dc45dfac1a026cbb4308f50b0121038f84e4eae1f7cac047e3ac51d03b8a346856c0576f6f9cadc778204f979f7b9fffffffff01102700000000000022002009b415a9b6f470f1460585bcccc0db3e9a2418aa93ed0420a42822ac7a12e383ffffffff


['0c8a77233e0fc561d6805e9fc9c859519b22644b9e8bffaa6dc44f9ae2e338a4']

**2.** Nun möchte Bob die an sein Skript gebundenen Satoshis wieder ausgeben. In dem Fall muss er **keine** Signatur oder der gleichen erzeugen. Zum Beweis das er berechtigt ist die Funds auszugeben muss er im Witnessfeld der Transaktion sein Secret und das Originalskript vorlegen.

In [15]:
alice_p2sh_locking_tx_parsed = alice.decoderawtransaction(alice.gettransaction(alice_p2wsh_locking_tx)["hex"])

bob_previous_tx_id_to_spent = alice_p2sh_locking_tx_parsed["txid"]

for tx_out in alice_p2sh_locking_tx_parsed["vout"]:
    if tx_out["value"] == Decimal("0.00010000"):
        bob_previous_tx_index_to_spent = tx_out["n"]
        bob_previous_script_pub_key_to_spent = tx_out["scriptPubKey"]["hex"]

#Transaktionsoutputdaten
bob_amount_to_spent = 9000
#bob's reciever address is alice public key

**2.1** Nun kann Bob die Transaktion erstellen um die eben von Alice erhaltenen Satoshis weiter zu versenden.

In [16]:
script_sig = Script([bytes.fromhex(bob_secret_hex), bytes.fromhex(bob_original_script_hex)])
transaction_input = CTxIn(bytes.fromhex(bob_previous_tx_id_to_spent), bob_previous_tx_index_to_spent, script_sig=Script([]), witness=script_sig.commands)

script_pubkey = p2pk_script(alice_public_key.sec_format())
transaction_output = CTxOut(bob_amount_to_spent, script_pubkey)

bob_spending_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=True)

print(f"Bob 'signierte' P2WSH Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")

TestShell().shutdown()

Bob 'signierte' P2WSH Spendingtransaktion: 010000000001011534d622c1381269ad49b64c91278888a68345180ed2496e817061f2d269d4df0000000000ffffffff0128230000000000002321038f84e4eae1f7cac047e3ac51d03b8a346856c0576f6f9cadc778204f979f7b9fac02125765206d616b652069742076697369626c6514125765206d616b652069742076697369626c6587ffffffff
2024-03-10T16:39:28.134000Z TestFramework (INFO): Stopping nodes
2024-03-10T16:39:28.237000Z TestFramework (INFO): Cleaning up /tmp/bitcoin_func_test_4vj6m9t2 on exit
2024-03-10T16:39:28.238000Z TestFramework (INFO): Tests successful
